### Cross Dataset Analysis 

#### The query compares the minimum, maximum, and average enrollment between the two datasets. 

In [4]:
%%bigquery

select 'clinical studies main' as dataset, MIN(csm.enrollment) as min_enrollment, MAX(csm.enrollment) as max_enrollment, AVG(csm.enrollment) as avg_enrollment
from nih_modeled.clinical_studies_main_Beam_DF csm 
where csm.lead_sponsor_agency not in (
    select distinct aero.sponsor 
    from aero_modeled.birds_eye_Beam_DF aero
    
    union all 
    
    select distinct csm.lead_sponsor_agency
    from nih_modeled.clinical_studies_main_Beam_DF csm 
    where csm.enrollment > 75000
    ) 

union all 

select 'aero bird\'s eye' as dataset, MIN(csm.enrollment) as min_enrollment, MAX(csm.enrollment) as max_enrollment, AVG(csm.enrollment) as avg_enrollment
from nih_modeled.clinical_studies_main_Beam_DF csm 
where csm.lead_sponsor_agency in (
    select distinct aero.sponsor 
    from aero_modeled.birds_eye_Beam_DF aero
    ) and (csm.enrollment <= 75000)

,dataset,min_enrollment,max_enrollment,avg_enrollment
0,aero bird's eye,0,61864,621.428066
1,clinical studies main,0,75000,346.656010


#### The query gathers the percentage of studies within each status category between the two datasets. The criteria for the status categories are those with greater than two percent of the total studies within the clinincal studies main dataset, therefore limiting it to seven categories, rather than several with smaller percentages. 

In [34]:
%%bigquery 

select 'clinical studies main' as dataset, overall_status, count(*) as count, ((count(*) / 234703) * 100) as percent 
from nih_modeled.clinical_studies_main_Beam_DF
group by overall_status 
having percent > 2

union all 

select 'aero bird\'s eye' as dataset, overall_status, count(*) as count, ((count(*) / 12799) * 100) as percent 
from nih_modeled.clinical_studies_main_Beam_DF csm inner join aero_modeled.birds_eye_Beam_DF aero
on csm.nct_number = aero.nct_number
where overall_status in (
    select overall_status
    from nih_modeled.clinical_studies_main_Beam_DF
    group by overall_status 
    having ((count(*) / 234703) * 100) > 2
    )
group by overall_status 

,dataset,overall_status,count,percent
0,aero bird's eye,Completed,9671,75.560591
1,aero bird's eye,Withdrawn,239,1.867333
2,aero bird's eye,Recruiting,897,7.008360
3,aero bird's eye,Terminated,1129,8.821002
4,aero bird's eye,Unknown status,5,0.039066
5,aero bird's eye,Not yet recruiting,136,1.062583
6,aero bird's eye,"Active, not recruiting",682,5.328541
7,clinical studies main,Completed,122231,52.079010
8,clinical studies main,Withdrawn,5166,2.201080
9,clinical studies main,Recruiting,40680,17.332544


#### The query gathers the percentage of studies within each condition category between the two datasets. The condions were limited to the top ten conditions in the studies within the birds eye dataset. 

In [82]:
%%bigquery 

(
select 'clinical studies main' as dataset, condition, count(*) as count, ((count(*) / 84800) * 100) as percent 
from nih_modeled.clinical_studies_main_Beam_DF 
where condition in (
    select condition
    from aero_modeled.birds_eye_Beam_DF
    group by condition 
    order by count(*) DESC
    limit 10
    )
group by condition 
order by count DESC
)

union all 

(
select 'aero bird\'s eye' as dataset, condition, count(*) as count, ((count(*) / 867) * 100) as percent 
from aero_modeled.birds_eye_Beam_DF
group by condition 
order by count DESC
limit 10
)

,dataset,condition,count,percent
0,aero bird's eye,"Diabetes Mellitus, Type 2",536,61.822376
1,aero bird's eye,Breast Neoplasms,388,44.752018
2,aero bird's eye,"Pulmonary Disease, Chronic Obstructive",339,39.100346
3,aero bird's eye,Hypertension,338,38.985006
4,aero bird's eye,Asthma,334,38.523645
5,aero bird's eye,"Arthritis, Rheumatoid",333,38.408304
6,aero bird's eye,"Influenza, Human",324,37.370242
7,aero bird's eye,Schizophrenia,292,33.679354
8,aero bird's eye,Diabetes Mellitus,270,31.141869
9,aero bird's eye,Alzheimer Disease,218,25.144175


### create views

In [83]:
%%bigquery

create or replace view reporting.v_avg_enrollment as
(
select 'clinical studies main' as dataset, MIN(csm.enrollment) as min_enrollment, MAX(csm.enrollment) as max_enrollment, AVG(csm.enrollment) as avg_enrollment
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
where csm.lead_sponsor_agency not in (
    select distinct aero.sponsor 
    from `probable-pager-266720.aero_modeled.birds_eye_Beam_DF` aero
    
    union all 
    
    select distinct csm.lead_sponsor_agency
    from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
    where csm.enrollment > 75000
    ) 

union all 

select 'aero bird\'s eye' as dataset, MIN(csm.enrollment) as min_enrollment, MAX(csm.enrollment) as max_enrollment, AVG(csm.enrollment) as avg_enrollment
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm 
where csm.lead_sponsor_agency in (
    select distinct aero.sponsor 
    from `probable-pager-266720.aero_modeled.birds_eye_Beam_DF` aero
    ) and (csm.enrollment <= 75000)
)

""


In [84]:
%%bigquery 

create or replace view reporting.v_status as
(
select 'clinical studies main' as dataset, overall_status, count(*) as count, ((count(*) / 234703) * 100) as percent 
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF`
group by overall_status 
having percent > 2

union all 

select 'aero bird\'s eye' as dataset, overall_status, count(*) as count, ((count(*) / 12799) * 100) as percent 
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF` csm inner join `probable-pager-266720.aero_modeled.birds_eye_Beam_DF` aero
on csm.nct_number = aero.nct_number
where overall_status in (
    select overall_status
    from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF`
    group by overall_status 
    having ((count(*) / 234703) * 100) > 2
    )
group by overall_status 
)

""


In [85]:
%%bigquery 

create or replace view reporting.v_conditions as
((
select 'clinical studies main' as dataset, condition, count(*) as count, ((count(*) / 84800) * 100) as percent 
from `probable-pager-266720.nih_modeled.clinical_studies_main_Beam_DF`
where condition in (
    select condition
    from `probable-pager-266720.aero_modeled.birds_eye_Beam_DF`
    group by condition 
    order by count(*) DESC
    limit 10
    )
group by condition 
order by count DESC
)

union all 

(
select 'aero bird\'s eye' as dataset, condition, count(*) as count, ((count(*) / 867) * 100) as percent 
from `probable-pager-266720.aero_modeled.birds_eye_Beam_DF`
group by condition 
order by count DESC
limit 10
))

""
